Building the SmartTag models
===


## Initial checks

In [ ]:
source .venv/bin/activate

In [ ]:
python setup.py install

In [ ]:
ls

In [ ]:
ls models

In [ ]:
ls data

In [ ]:
ls data4th

In [ ]:
smtag-neo2xml --help

In [ ]:
smtag-convert2th --help

In [ ]:
smtag-meta --help

In [ ]:
smtag-eval --help

In [ ]:
smtag-predict --help

## Preparation of corpus of xml documents and images

In [ ]:
smtag-neo2xml -l10000 -f 181203all # on large sd-graph with 30000 panels from 1100 papers

In [ ]:
smtag-ocr # run only once

## Small molecules

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@type='molecule']" \
-f 10X_L1200_molecule -w /efs/smtag --noocr

In [ ]:
smtag-meta -f 10X_L1200_molecule -w /efs/smtag \
-E120 -Z128 -R0.01 \
-o small_molecule

saved 10X_L1200_molecule_small_molecule_2018-12-22-19-52.sddl (size: 96117)

In [ ]:
smtag-eval -f .zip -S -T

In [ ]:
smtag-eval -f 5X_L1200_molecule -m 5X_L1200_molecule_small_molecule_2018-12-19-06-04.zip

---

In [ ]:
cp models/ rack/small_molecule.zip

## Gene products

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X5 -y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-f 5X_L1200_geneprod --noocr -w /efs/smtag


In [ ]:
smtag-meta -f 5X_L1200_geneprod -w /efs/smtag \
-E120 -Z32 -R0.01 \
-o geneprod

model saved under `5X_L1200_geneprod_geneprod_2018-12-22-18-16.sddl`

In [ ]:
smtag-eval -f 3X_L600_geneprod -m 3X_L600_geneprod_geneprod_2018-12-05-00-32.zip -S -T

In [ ]:
smtag-eval -f .zip

In [ ]:
cp models/3X_L600_geneprod_geneprod_2018-12-05-00-32.zip rack/geneprod.zip

## Subcellular

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@type='subcellular']" \
-f 10X_L1200_subcellular -w /efs/smtag --noocr

In [ ]:
smtag-meta -f 10X_L1200_subcellular -w /efs/smtag \
-E120 -Z128 -R0.01 \
-o subcellular

save as `10X_L1200_subcellular_subcellular_2018-12-22-21-12.sddl` (size: 100167)`

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

In [ ]:
cp models/.zip rack/subcellular.zip

## Cell

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@type='cell']" \
-f 10X_L1200_cell -w /efs/smtag --noocr

In [ ]:
smtag-meta -f 10X_L1200_cell \
-E120 -Z128 -R0.01 \
-o cell

saved `` (size: 100167)

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

In [ ]:
cp models/.zip rack/cell.zip

## Tissue 

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@type='tissue']" \
-f 10X_L1200_tissue --noocr

In [ ]:
smtag-meta -f 3X_L600_tissue \
-E120 -Z128 -R0.01 \
-o tissue

saved `3X_L600_tissue_tissue_2018-12-06-23-58.sddl` (size: 100167)

In [ ]:
cp models/3X_L600_tissue_tissue_2018-12-06-23-58.zip rack/tissue.zip

In [ ]:
smtag-eval -f 3X_L600_tissue -m 3X_L600_tissue_tissue_2018-12-06-23-58.zip -S -T

In [ ]:
smtag-eval -f 3X_L600_tissue -m 3X_L600_tissue_tissue_2018-12-06-23-58.zip

## Organism 

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@type='organism']" \
-f 3X_L600_organism

In [ ]:
smtag-meta -f 10X_L1200_organism \
-E120 -Z128 -R0.01 \
-o organism

saved `` (size: 100167)

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

In [ ]:
cp models/ rack/organism.zip

## Diseases 

In [ ]:
smtag-convert2th -L1200 -X10 -b -c NCBI_disease -f 5X_L1200_NCBI_disease

In [ ]:
smtag-meta -E120 -Z128 -R0.01 -o disease -f 10X_L1200_NCBI_disease

saved `` (size: 100167)

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

In [ ]:
cp models/.zip rack/disease.zip

## Experimental assay 

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X10 -y ".//sd-tag[@category='assay']" \
-f 10X_L1200_assay

In [ ]:
smtag-meta -f 10X_L1200_assay \
-E120 -Z128 -R0.01 \
-o assay

saved `` (size: 100167)

In [ ]:
cp models/.zip rack/exp_assay.zip

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

## Intervention-assay geneprod 

In [ ]:
smtag-convert2th -c 181203all \
-L1200 -X10 \
-E ".//figure-caption" \
-y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-e ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-A ".//sd-tag[@role='intervention']",\
".//sd-tag[@role='assayed']",\
".//sd-tag[@role='normalizing']",\
".//sd-tag[@role='experiment']",\
".//sd-tag[@role='component']", \
-f 10X_L1200_geneprod_anonym_not_reporter -w /efs/smtag --noocr

In [ ]:
smtag-meta -f 10X_L1200_geneprod_anonym_not_reporter -w /efs/smtag \
-E120 -Z128 -R0.01 \
-o intervention,assayed

saved `10X_L1200_geneprod_anonym_not_reporter_intervention_assayed_2018-12-22-22-40.sddl` (size: 96257)

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f  -m .zip

In [1]:
cp models/.zip rack/role_geneprod.zip

cp: models/.zip: No such file or directory


: 1

## Geneprod Reporter

In [ ]:
smtag-convert2th -c 181203all \
-E ".//figure-caption" \
-L1200 -X5 \
-y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-e ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-f 5X_L1200_geneprod_reporter -w /efs/smtag --noocr

In [ ]:
smtag-meta -f 5X_L1200_geneprod_reporter -w /efs/smtag \
-E120 -Z128 -R0.01 \
-o reporter

saved `5X_L1200_geneprod_reporter_reporter_2018-12-22-23-44.sddl` (size: 96117)

In [ ]:
smtag-eval -f .zip -S -T

In [ ]:
smtag-eval -f 5X_L1200_geneprod_reporter -m 

In [ ]:
cp models/.zip rack/reporter_geneprod.zip

## Intervention-assay small molecule

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 \
-y ".//sd-tag[@type='molecule']" \
-e ".//sd-tag[@type='molecule']" \
-A ".//sd-tag[@type='molecule']" \
-f 3X_L600_molecule_anonym

In [ ]:
smtag-meta -f 3X_L600_molecule_anonym \
-E120 -Z128 -R0.01 \
-o intervention,assayed

saved `3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.sddl` (size: 100315)

In [ ]:
cp models/3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip rack/role_small_molecule.zip

In [ ]:
smtag-eval -f 3X_L600_molecule_anonym -m 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip -S -T

In [ ]:
smtag-eval -f 3X_L600_molecule_anonym -m 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip

## Panel start

In [ ]:
smtag-convert2th -c 181203all \
--noocr -E ".//figure-caption" \
-L1200 -X10 -f 10X_L1200_figure_level_no_ocr

In [ ]:
smtag-meta -f 10X_L1200_figure_level_no_ocr \
-E120 -Z128 -R0.01 \
-o panel_start

saved `` (size: 100167)

In [ ]:
smtag-eval -f  -m .zip -S -T

In [ ]:
smtag-eval -f -m .zip

In [ ]:
cp models/zip rack/panel_start.zip

## Special models (experimental)

### Whole document level intervention-assayed model

In [ ]:
smtag-convert2th -c 181203all \
--noocr -E "." \
-e ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-A ".//sd-tag[@role='intervention']",\
".//sd-tag[@role='assayed']",\
".//sd-tag[@role='normalizing']",\
".//sd-tag[@role='experiment']",\
".//sd-tag[@role='component']", \
-L6000 -X -f 1X_L5000_doc_level_geneprod_anonym_not_reporter_no_ocr

In [ ]:
smtag-meta -f 1X_L5000_doc_level_geneprod_anonym_not_reporter_no_ocr \
-E200 -Z32 -R0.01 \
-n 8,8,8,8 -p 2,2,2,2 -k 6,6,6,6 \
-o intervention,assayed

In [ ]:
smtag-convert2th -c 181203all \
--noocr -E "." \
-y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-L6000 -X3 -f 3X_L6000_doc_level_geneprod_no_ocr

In [ ]:
smtag-meta -f 3X_L6000_doc_level_geneprod_no_ocr \
-E200 -Z32 -R0.01 \
-n 8,8,8,8 -p 2,2,2,2 -k 6,6,6,6 \
-o geneprod

## Rack for SmartTag engine

In [ ]:
ls rack

## Hyperparam scans

In [ ]:
smtag-meta -E50 -o panel_start -H depth,kernel -I 25 -f 5X_L1200_entities -w /efs/smtag # after data transfer to GPU machine

Winning combo:
    4,4,4,4
    
But a lot of variability. Initialization issues?

In [ ]:
smtag-meta -E50 -o geneprod -H kernel,depth -I 25 -f 5X_L1200_gene_protein -w /efs/smtag # after transfer to GPU machine

Winning combo with `f1=0.80` after 50 epochs, no overfitting with valid:

    namebase=5X_L1200_gene_protein; modelname=; learning_rate=0.01000000000000001; epochs=50; minibatch_size=32; selected_features=['geneprod']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; dropout=0.1; validation_fraction=0.2; nf_input=32; nf_output=1